# Modelle bewerten mit dem `scoring`-Parameter und Cross-Validation

In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Daten importieren
heart_disease = pd.read_csv("data/heart-disease.csv")

# Feature-Matrix und Label-Vektor:
X = heart_disease.drop("target", axis=1)
y = heart_disease["target"]

# Modell erstellen:
clf = RandomForestClassifier(random_state=42)

# Cross-validation anwenden:
cv_acc = cross_val_score(clf, X, y, cv=5, scoring=None) # Teilung der Daten in 5 gleiche Teile (Folds)
cv_acc

array([0.81967213, 0.8852459 , 0.83606557, 0.85      , 0.8       ])

`Gesamtdaten = [Fold 1] [Fold 2] [Fold 3] [Fold 4] [Fold 5]`

| Durchgang | Trainingsdaten                | Testdaten |
| --------- | ----------------------------- | --------- |
| 1         | Fold2 + Fold3 + Fold4 + Fold5 | Fold1     |
| 2         | Fold1 + Fold3 + Fold4 + Fold5 | Fold2     |
| 3         | Fold1 + Fold2 + Fold4 + Fold5 | Fold3     |
| 4         | Fold1 + Fold2 + Fold3 + Fold5 | Fold4     |
| 5         | Fold1 + Fold2 + Fold3 + Fold4 | Fold5     |


$$\text{Accuracy}=\frac{\text{Anzahl korrekt vorhergesagter Klassen}}{\text{Gesamtanzahl}}$$

In [3]:
mean_accuracy = cv_acc.mean()
print(f"Durchschnitt der Accuracy: {mean_accuracy:.4f}")

Durchschnitt der Accuracy: 0.8382


In [4]:
std_accuracy = cv_acc.std()
print(f"Standardabweichung der Accuracy: {std_accuracy:.4f}")

Standardabweichung der Accuracy: 0.0288


Anzahl der Fold erhöhen:

In [5]:
cv_acc = cross_val_score(clf, X, y, cv=50, scoring=None) 

mean_accuracy = cv_acc.mean()
print(f"Durchschnitt der Accuracy: {mean_accuracy:.4f}")

std_accuracy = cv_acc.std()
print(f"Standardabweichung der Accuracy: {std_accuracy:.4f}")

Durchschnitt der Accuracy: 0.8176
Standardabweichung der Accuracy: 0.1600


Mit mehreren Metriken:

In [6]:
from sklearn.model_selection import cross_validate

scoring = ["accuracy", "precision", "recall", "f1"]

scores = cross_validate(
    estimator=RandomForestClassifier(random_state=42),
    X=X,
    y=y,
    cv=5,
    scoring=scoring
)

df_scores = pd.DataFrame(scores)
df_scores

,fit_time,score_time,test_accuracy,test_precision,test_recall,test_f1
0,0.112228,0.017497,0.819672,0.823529,0.848485,0.835821
1,0.105932,0.012918,0.885246,0.906250,0.878788,0.892308
2,0.108178,0.011316,0.836066,0.848485,0.848485,0.848485
3,0.107314,0.011802,0.850000,0.875000,0.848485,0.861538
4,0.110251,0.011597,0.800000,0.783784,0.878788,0.828571


In [7]:
mean_row = df_scores.mean()
mean_row

fit_time          0.108781
score_time        0.013026
test_accuracy     0.838197
test_precision    0.847410
test_recall       0.860606
test_f1           0.853345
dtype: float64

In [8]:
std_row = df_scores.std()
std_row

fit_time          0.002484
score_time        0.002572
test_accuracy     0.032248
test_precision    0.047019
test_recall       0.016598
test_f1           0.025148
dtype: float64

In [9]:
df_mean_std = pd.DataFrame([mean_row, std_row], index=["mean", "std"])
df_scores = pd.concat([df_scores, df_mean_std])
df_scores

,fit_time,score_time,test_accuracy,test_precision,test_recall,test_f1
0,0.112228,0.017497,0.819672,0.823529,0.848485,0.835821
1,0.105932,0.012918,0.885246,0.906250,0.878788,0.892308
2,0.108178,0.011316,0.836066,0.848485,0.848485,0.848485
3,0.107314,0.011802,0.850000,0.875000,0.848485,0.861538
4,0.110251,0.011597,0.800000,0.783784,0.878788,0.828571
mean,0.108781,0.013026,0.838197,0.847410,0.860606,0.853345
std,0.002484,0.002572,0.032248,0.047019,0.016598,0.025148


In [10]:
new_index = []
for i in range(5):
    new_index.append(f"Fold{i + 1}")

new_index.extend(["mean", "std"])

In [11]:
df_scores.index = new_index
df_scores

,fit_time,score_time,test_accuracy,test_precision,test_recall,test_f1
Fold1,0.112228,0.017497,0.819672,0.823529,0.848485,0.835821
Fold2,0.105932,0.012918,0.885246,0.906250,0.878788,0.892308
Fold3,0.108178,0.011316,0.836066,0.848485,0.848485,0.848485
Fold4,0.107314,0.011802,0.850000,0.875000,0.848485,0.861538
Fold5,0.110251,0.011597,0.800000,0.783784,0.878788,0.828571
mean,0.108781,0.013026,0.838197,0.847410,0.860606,0.853345
std,0.002484,0.002572,0.032248,0.047019,0.016598,0.025148


In [12]:
import plotly.graph_objects as go

cv_acc = cross_val_score(clf, X, y, cv=10)
mean_accuracy = cv_acc.mean()
std_accuracy = cv_acc.std()

fig1 = go.Figure()

# Accuracy pro Fold:
fig1.add_trace(go.Bar(
    x=[f"Fold {i+1}" for i in range(len(cv_acc))],
    y=cv_acc,
    name="Accuracy pro Fold",
    marker_color="steelblue"
))

# Durchschnittslinie:
fig1.add_trace(go.Scatter(
    x=[f"Fold {i+1}" for i in range(len(cv_acc))],
    y=[mean_accuracy] * len(cv_acc),
    mode="lines",
    name=f"Durchschnitt: {mean_accuracy:.3f}",
    line=dict(
        color="red",
        dash="dash"
    )
))

# +std-Linie:
fig1.add_trace(go.Scatter(
    x=[f"Fold {i+1}" for i in range(len(cv_acc))],
    y=[mean_accuracy + std_accuracy] * len(cv_acc),
    mode="lines",
    name=f"+1 Std: {(mean_accuracy + std_accuracy):.3f}",
    line=dict(color="orange", dash="dot")
))

# -std-Linie:
fig1.add_trace(go.Scatter(
    x=[f"Fold {i+1}" for i in range(len(cv_acc))],
    y=[mean_accuracy - std_accuracy] * len(cv_acc),
    mode="lines",
    name=f"-1 Std: {(mean_accuracy - std_accuracy):.3f}",
    line=dict(color="orange", dash="dot"),
    fill="tonexty",
    fillcolor="rgba(255, 165, 0, 0.2)"
))

fig1.update_layout(
    title="Cross-Validation pro Fold",
    xaxis_title="Fold",
    yaxis_title="Accuracy",
    yaxis=dict(range=[0, 1])
)
